In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
import os
import tensorflow as tf
from keras.utils import to_categorical
from keras.utils import load_img, img_to_array
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten,Conv2D, BatchNormalization,Activation,MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam,SGD,RMSprop
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder


In [20]:
import os
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.optimizers import Adam, Adamax
#from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications import EfficientNetB3

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  0


In [3]:
train_dir = 'Data/train'
test_dir = 'Data/test'

In [4]:
def createdataFrame(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)

        print(label,'done')
    return image_paths,labels


In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataFrame(train_dir)

angry done
disgusted done
fearful done
happy done
neutral done
sad done
surprised done


In [6]:
print(train)


                                image      label
0            Data/train\angry\im0.png      angry
1            Data/train\angry\im1.png      angry
2           Data/train\angry\im10.png      angry
3          Data/train\angry\im100.png      angry
4         Data/train\angry\im1000.png      angry
...                               ...        ...
28704  Data/train\surprised\im995.png  surprised
28705  Data/train\surprised\im996.png  surprised
28706  Data/train\surprised\im997.png  surprised
28707  Data/train\surprised\im998.png  surprised
28708  Data/train\surprised\im999.png  surprised

[28709 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataFrame(test_dir)

angry done
disgusted done
fearful done
happy done
neutral done
sad done
surprised done


In [9]:
print(test)

                             image      label
0          Data/test\angry\im0.png      angry
1          Data/test\angry\im1.png      angry
2         Data/test\angry\im10.png      angry
3        Data/test\angry\im100.png      angry
4        Data/test\angry\im101.png      angry
...                            ...        ...
7173  Data/test\surprised\im95.png  surprised
7174  Data/test\surprised\im96.png  surprised
7175  Data/test\surprised\im97.png  surprised
7176  Data/test\surprised\im98.png  surprised
7177  Data/test\surprised\im99.png  surprised

[7178 rows x 2 columns]


In [10]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode="grayscale")
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features 

In [11]:
train_features = extract_features(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

In [12]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0


In [14]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [16]:
def model_performance(history, Epochs):
    # Define needed variables
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
    Epochs = [i+1 for i in range(len(tr_acc))]
    
    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')
    
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout
    plt.show()
    

# Evaluate the model
def model_evaluation(model):
    train_score = model.evaluate(train_gen, verbose= 1)
    valid_score = model.evaluate(valid_gen, verbose= 1)
    test_score = model.evaluate(test_gen, verbose= 1)
    
    print("Train Loss: ", train_score[0])
    print("Train Accuracy: ", train_score[1])
    print('-' * 20)
    print("Validation Loss: ", valid_score[0])
    print("Validation Accuracy: ", valid_score[1])
    print('-' * 20)
    print("Test Loss: ", test_score[0])
    print("Test Accuracy: ", test_score[1])
    

# Get Predictions
def get_pred(model, test_gen):
    
    preds = model.predict(test_gen)
    y_pred = np.argmax(preds, axis = 1)
    
    return y_pred


# Confusion Matrix
def plot_confusion_matrix(test_gen, y_pred):
    
    g_dict = test_gen.class_indices
    classes = list(g_dict.keys())
    
    # Display the confusion matrix
    cm = confusion_matrix(test_gen.classes, y_pred)

    plt.figure(figsize= (10, 10))
    plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation= 45)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
    
    
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.show()
    

In [21]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Add, Concatenate, Input, Dropout
from keras.layers import Layer
import tensorflow as tf

# Define the Graph Convolutional Layer
class GraphConvLayer(Layer):
    def __init__(self, output_dim, activation=None, **kwargs):
        self.output_dim = output_dim
        self.activation = activation
        super(GraphConvLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][-1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(GraphConvLayer, self).build(input_shape)

    def call(self, inputs):
        X, A = inputs
        output = tf.matmul(A, X)
        output = tf.matmul(output, self.kernel)
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

def residual_block(x, filters):
    y = Conv2D(filters, kernel_size=(3, 3), padding="same", activation="relu")(x)
    y = BatchNormalization()(y)
    y = Conv2D(filters, kernel_size=(3, 3), padding="same", activation="relu")(y)
    y = BatchNormalization()(y)
    residual = Conv2D(filters, kernel_size=(1, 1), padding="same")(x)  
    y = Add()([residual, y])
    y = MaxPooling2D()(y) 
    return y

def inception_module(x, filters):
    branch1x1 = Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    
    branch3x3 = Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv2D(filters, (3, 3), padding='same', activation='relu')(branch3x3)
    
    branch5x5 = Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv2D(filters, (5, 5), padding='same', activation='relu')(branch5x5)
    
    branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = Conv2D(filters, (1, 1), padding='same', activation='relu')(branch_pool)
    
    outputs = [branch1x1, branch3x3, branch5x5, branch_pool]
    return Concatenate(axis=-1)(outputs)

def dense_block(x, units, dropout_rate):
    x = Dense(units, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    return x

# Define number of nodes in the graph and number of classes
num_nodes = 64  
class_counts = 10  

input_layer = Input(shape=(224, 224, 3)) 
x = Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu")(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# Residual blocks
x = residual_block(x, 32)
x = residual_block(x, 64)
x = residual_block(x, 128)
x = residual_block(x, 256)
x = residual_block(x, 512)

# Inception modules
x = inception_module(x, 256)
x = inception_module(x, 128)
x = inception_module(x, 64)

# Additional convolutional layers
x = Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# Flatten layer
x = Flatten()(x)

# Define adjacency matrix placeholder
A = Input(shape=(num_nodes, num_nodes))

# GCN Layer 1
gcn_output = GraphConvLayer(output_dim=64, activation='relu')([x, A])

# GCN Layer 2
gcn_output = GraphConvLayer(output_dim=32, activation='relu')([gcn_output, A])

# Flatten GCN output
gcn_output_flatten = Flatten()(gcn_output)


concatenated_output = Concatenate()([x, gcn_output_flatten])

# Dense layers
x = dense_block(concatenated_output, 512, 0.5) 
x = dense_block(x, 256, 0.3)   
x = dense_block(x, 128, 0.2)   

# Output layer
output_layer = Dense(class_counts, activation="sigmoid")(x)

# Create model
cnn_gcn_model = Model(inputs=[input_layer, A], outputs=output_layer)

In [22]:
cnn_gcn_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

cnn_gcn_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │      9,248 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │      1,056 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 112, 112,  │          0 │ conv2d_3[0][0],   │
│                     │ 32)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ add[0][0]         │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 56, 56,    │      2,112 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 56, 56,    │          0 │ conv2d_6[0][0], 

 Total params: 19,046,314 (72.66 MB)

 Trainable params: 19,041,258 (72.64 MB)

 Non-trainable params: 5,056 (19.75 KB)

In [23]:
epochs = 20


#cnn_gcn_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)
cnn_gcn_model.fit(x= np.array(x_train),y= np.array(y_train),batch_size=128, epochs=100,validation_data=(x_test,y_test))

Epoch 1/100


ValueError: Layer 'functional_1' expected 2 input(s). Received 1 instead.